In [1]:
import torch
import torch.nn as nn

In [2]:
class BasicConv(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, **kwargs):
        super(BasicConv, self).__init__()

        self.conv_block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=kernel_size, bias=False, **kwargs),
            nn.BatchNorm2d(out_ch),
            nn.ReLU6(inplace=True) #Relu랑 같은데 상한이 6으로 제한된 레이어
            # 기존 Relu보다 고정 소수점 연산(fixed-point arithmetic)에 더 유리함
            # 따라서 모바일 및 임베디드 디바이스에 대하여 유리함
        )

    def forward(self, x):
        x = self.conv_block(x)

        return x

In [3]:
class DepthSep(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super(DepthSep, self).__init__()

        self.depthwise = BasicConv(in_ch, in_ch, kernel_size=3, stride=stride, padding=1,
                                   groups = in_ch)
        # 여기서 groups 입력 채널과 출력 채널사이의 관계를 나타냄
        # default=1 => 모든 입력은 모든 출력과 conv 연산이 됨
        # 2, 3, 4 => 입력을 2, 3, 4 그룹으로 나누어서 각각 conv연산 후 concat
        # group = in_ch --> 이게 Depthwise의 `Separable`에 해당하는 항목임

        self.pointwise = BasicConv(in_ch, out_ch, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)

        return x

In [4]:
class MobileNetV1(nn.Module):
    def __init__(self, width_multiplier, num_classes=1000, init_weight=True):
        super(MobileNetV1, self).__init__()

        self.alpha = width_multiplier #네트워크 각 층의 필터 개수를 조정하는 인자값

        self.stem = BasicConv(3, int(32*self.alpha), kernel_size=3, stride=2, padding=1)

        self.feature_ext = nn.Sequential(
            DepthSep(int(32*self.alpha), int(64*self.alpha)),
            DepthSep(int(64*self.alpha), int(128*self.alpha), stride=2),
            DepthSep(int(128*self.alpha), int(128*self.alpha)),
            DepthSep(int(128*self.alpha), int(256*self.alpha), stride=2),
            DepthSep(int(256*self.alpha), int(256*self.alpha)),
            DepthSep(int(256*self.alpha), int(512*self.alpha), stride=2),
            DepthSep(int(512*self.alpha), int(512*self.alpha)),
            DepthSep(int(512*self.alpha), int(512*self.alpha)),
            DepthSep(int(512*self.alpha), int(512*self.alpha)),
            DepthSep(int(512*self.alpha), int(512*self.alpha)),
            DepthSep(int(512*self.alpha), int(512*self.alpha)),
            DepthSep(int(512*self.alpha), int(1024*self.alpha), stride=2),
            DepthSep(int(1024*self.alpha), int(1024*self.alpha))
        )

        self.classfier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(int(1024*self.alpha), num_classes)
        )

        if init_weight: #초기화 구동함수 호출
            self._initialize_weight()

    def forward(self, x):
        x = self.stem(x)
        x = self.feature_ext(x)
        x = self.classfier(x)

        return x
    
    #모델의 초기 Random을 커스터마이징 하기 위한 함수
    def _initialize_weight(self):
        for m in self.modules(): #설계한 모델의 모든 레이어를 순회
            if isinstance(m, nn.Conv2d): #conv의 파라미터(weight, bias)의 초가깂설정
                # Kaiming 초기화를 사용한 이유:
                # Kaiming 초기화는 ReLU 활성화 함수와 함께 사용될 때 좋은 성능을 보임
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            
            elif isinstance(m, nn.BatchNorm2d): #BN의 파라미터(weight, bias)의 초가깂설정
                # BatchNorm 레이어의 가중치와 바이어스를 간단한 값으로 초기화
                nn.init.constant_(m.weight, 1) # 1로 다 채움
                nn.init.constant_(m.bias, 0) # 0으로 다 채움

            elif isinstance(m, nn.Linear): #FCL의 파라미터(weight, bias)의 초기값 설정
                # 선형 레이어의 가중치를 정규 분포로 초기화
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


In [5]:
m_keys = ['W_100%', 'W_75%', 'W_50%'] #모델에 Width Multiplier
d_keys = ['R_224', 'R_192', 'R_128'] #데이터셋에 Resolution Multiplier

In [6]:

import copy

model_100 = MobileNetV1(width_multiplier=1.0, num_classes=10)
model_75 = MobileNetV1(width_multiplier=0.75, num_classes=10)
model_50 = MobileNetV1(width_multiplier=0.5, num_classes=10)

models = {
    'W_100%' : model_100.to('cpu'),
    'W_75%' : model_75.to('cpu'),
    'W_50%' : model_50.to('cpu'),
}

# 모델의 초기 가중치 저장
initial_weights = {
    'W_100%': copy.deepcopy(model_100.state_dict()),
    'W_75%': copy.deepcopy(model_75.state_dict()),
    'W_50%': copy.deepcopy(model_50.state_dict())
}

In [7]:
import os
from torchvision import datasets

root = './data/Animals-10'

# 원래 img_dataset 초기화
original_img_dataset = {}

original_img_dataset['train'] = datasets.ImageFolder(os.path.join(root, 'train'))
original_img_dataset['val'] = datasets.ImageFolder(os.path.join(root, 'val'))

In [8]:
# 데이터 전처리 방법론 정의
from torchvision.transforms import v2

animals_val = {'mean' : [0.5177, 0.5003, 0.4126],
                'std' : [0.2133, 0.2130, 0.2149]
}

def define_transform(img_size, normal_val, augment=False):
    transform_list = []

    if augment:
        transform_list += [ #데이터 증강은 반전, 색상밝기채도, 아핀 3가지
            v2.RandomHorizontalFlip(p=0.5),
            v2.ColorJitter(brightness=0.4,
                            contrast=0.4,
                            saturation=0.4,
                            hue=0.1),
            v2.RandomAffine(degrees=(30, 70),
                            translate=(0.1, 0.3),
                            scale=(0.5, 0.75)),
        ]

    transform_list += [
        v2.Resize((img_size, img_size)), #이미지 사이즈별로 리사이징
        v2.ToImage(),  #이미지를 Tensor 자료형으로 변환
        v2.ToDtype(torch.float32, scale=True), #텐서 자료형을 [0~1]로 정규화
        v2.Normalize(mean=normal_val['mean'], std=normal_val['std']) #데이터셋 표준화
    ]

    return v2.Compose(transform_list)

In [9]:
train_transfroms = {
    'R_224' : define_transform(224, animals_val, augment=True),
    'R_192' : define_transform(192, animals_val, augment=True),
    'R_128' : define_transform(128, animals_val, augment=True)
}

val_transfroms = {
    'R_224' : define_transform(224, animals_val, augment=False),
    'R_192' : define_transform(192, animals_val, augment=False),
    'R_128' : define_transform(128, animals_val, augment=False)
}

In [10]:
# img_dataset 딥카피
img_dataset = {
    'R_224': copy.deepcopy(original_img_dataset),
    'R_192': copy.deepcopy(original_img_dataset),
    'R_128': copy.deepcopy(original_img_dataset),
}

for d_key in img_dataset:
    img_dataset[d_key]['train'].transform = train_transfroms[d_key]
    img_dataset[d_key]['val'].transform = val_transfroms[d_key]

In [11]:
from torch.utils.data import DataLoader

bs = 256 #배치사이즈 크기

#데이터로더 생성
dataloaders_dict = {
    'R_224': None,
    'R_192': None,
    'R_128': None,
}

for d_key in dataloaders_dict:
    dataloaders_dict[d_key] = {
        'train' : DataLoader(img_dataset[d_key]['train'], batch_size=bs, shuffle=True),
        'val' : DataLoader(img_dataset[d_key]['val'], batch_size=bs, shuffle=False),
    }

In [12]:
#GPU사용 가능여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import torch.optim as optim

# 모델을 GPU로 이전
for m_key in models:
    models[m_key].to(device)

#손실함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()

optimizers = {
    'W_100%': optim.Adam(models['W_100%'].parameters(), lr=0.001),
    'W_75%': optim.Adam(models['W_75%'].parameters(), lr=0.001),
    'W_50%': optim.Adam(models['W_50%'].parameters(), lr=0.001)
}

In [13]:
# 사전에 모듈화 한 학습/검증용 라이브러리 import
from C_ModelTrainer import ModelTrainer

epoch_step = 3 #특정 epoch마다 모델의 훈련/검증 정보 출력
# BC_mode = True : 이진분류 문제 풀이 , BC_mode = False : 다중분류 문제 풀이
trainer = ModelTrainer(epoch_step=epoch_step, device=device.type, BC_mode=False)

In [14]:
# 학습과 검증 손실 및 정확도를 저장할 딕셔너리
d_keys = dataloaders_dict.keys()
m_keys = models.keys()
history = {mk: {dk: {'loss': [], 'accuracy': []} for dk in d_keys} for mk in m_keys}

num_epoch = 1

In [15]:
# 모델 훈련 및 평가
for d_key in dataloaders_dict:
    dataloaders = dataloaders_dict[d_key]
    for m_key in models:
        model = models[m_key]
        optimizer = optimizers[m_key]

        #학습/검증 epoch 수행 전 모델 파라미터 초기화
        model.load_state_dict(initial_weights[m_key])

        for epoch in range(num_epoch):
            # 훈련 손실과 훈련 성과지표를 반환 받습니다.
            train_loss, train_acc = trainer.model_train(model, dataloaders['train'], 
                                                        criterion, optimizer, epoch)

            # 검증 손실과 검증 성과지표를 반환 받습니다.
            test_loss, test_acc = trainer.model_evaluate(model, dataloaders['val'], 
                                                         criterion, epoch)

            # 손실과 성능지표를 리스트에 저장
            history[m_key][d_key]['loss'].append((train_loss, test_loss))
            history[m_key][d_key]['accuracy'].append((train_acc, test_acc))

            # epoch가 특정 배수일 때만 출력하기
            if (epoch + 1) % epoch_step == 0 or epoch == 0:
                print(f"epoch {epoch+1:03d}," + "\t" + 
                      f"[{m_key}, {d_key}] 훈련 [Loss: {train_loss:.3f}, " +
                      f"Acc: {train_acc*100:.2f}%]")
                print(f"epoch {epoch+1:03d}," + "\t" + 
                      f"[{m_key}, {d_key}] 검증 [Loss: {test_loss:.3f}, " +
                      f"Acc: {test_acc*100:.2f}%]")

100%|██████████| 16/16 [00:11<00:00,  1.43it/s]


epoch 001,	[W_100%, R_224] 훈련 [Loss: 2.114, Acc: 22.87%]
epoch 001,	[W_100%, R_224] 검증 [Loss: 2.900, Acc: 19.10%]


100%|██████████| 16/16 [00:10<00:00,  1.49it/s]


epoch 001,	[W_75%, R_224] 훈련 [Loss: 2.166, Acc: 21.09%]
epoch 001,	[W_75%, R_224] 검증 [Loss: 2.434, Acc: 27.00%]


100%|██████████| 16/16 [00:10<00:00,  1.49it/s]


...

epoch 001,	[W_100%, R_128] 훈련 [Loss: 2.170, Acc: 21.04%]
epoch 001,	[W_100%, R_128] 검증 [Loss: 2.299, Acc: 21.75%]


100%|██████████| 16/16 [00:06<00:00,  2.39it/s]


epoch 001,	[W_75%, R_128] 훈련 [Loss: 2.191, Acc: 19.93%]
epoch 001,	[W_75%, R_128] 검증 [Loss: 2.246, Acc: 20.27%]


100%|██████████| 16/16 [00:06<00:00,  2.49it/s]

epoch 001,	[W_50%, R_128] 훈련 [Loss: 2.187, Acc: 19.50%]
epoch 001,	[W_50%, R_128] 검증 [Loss: 2.165, Acc: 21.88%]


In [16]:
from thop import profile
from thop import clever_format

input_tensors = {
    'R_224' : torch.randn(1, 3, 224, 224).to(device),
    'R_192' : torch.randn(1, 3, 192, 192).to(device),
    'R_128' : torch.randn(1, 3, 128, 128).to(device),
}

for d_key in models:
    for m_key in input_tensors:
        # FLOPs 및 파라미터 수 계산 -> 여기서 inputs는 `튜플` 자료형이어야만 한다!!
        flops, params = profile(models[d_key], inputs=(input_tensors[m_key], ))

        # 보기 좋은 형식으로 출력
        flops, params = clever_format([flops, params], "%.3f")
        print(f"[{d_key}, {m_key}] FLOPs: {flops}, Params: {params}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU6'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[W_100%, R_224] FLOPs: 587.949M, Params: 3.217M
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU6'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <cla